# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = get_date_list(data)
data = read_data(data)

Date List Created
Record num : 129543
Wall time: 13.4 s


In [3]:
%%time
data['df']['tick']      = (data["df"]['Ask'] + data["df"]['Bid'])/2
data['df']['DateTime_frmt']   = [dt.datetime.strptime(x.split(".")[0],"%Y%m%d %H:%M:%S") for x in data["df"]['DateTime']]
data['df_ohlc'] = data['df'][['DateTime_frmt', 'tick']]

Wall time: 1.15 s


In [4]:
data = get_ohlc(data)

In [5]:
data['df_ohlc']['cdl_hammer'] = talib.CDLHAMMER(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])
data['df_ohlc']['cdl_engulfing'] = talib.CDLENGULFING(data['df_ohlc']['open'], data['df_ohlc']['high'], data['df_ohlc']['low'], data['df_ohlc']['close'])

In [6]:
data = merge_ohlc_data(data)

100%|████████████████████████████████████████| 288/288 [00:03<00:00, 89.13it/s]


In [7]:
data['df']['cdl_hammer'] = data['df']['cdl_hammer'].replace({0:np.nan})
data['df']['cdl_hammer'] = np.where(data['df']['cdl_hammer'] == 100, data['df']['tick'], data['df']['cdl_hammer'])

data['df']['cdl_engulfing'] = data['df']['cdl_engulfing'].replace({0:np.nan})
data['df']['cdl_engulfing'] = np.where(data['df']['cdl_engulfing'] == 100, data['df']['tick'], data['df']['cdl_engulfing'])
# Temp step
data['df']['cdl_engulfing'] = np.where(data['df']['cdl_engulfing'] == -100, data['df']['tick'], data['df']['cdl_engulfing'])

In [8]:
data['df'][data['df']['cdl_engulfing'].notnull()]

,DateTime,Bid,Ask,Volume,i,tick,DateTime_frmt,open,high,low,close,cdl_hammer,cdl_engulfing
1050,20220720 00:24:59.108,1.02398,1.02405,1,0,1.024015,2022-07-20 00:24:59,1.024010,1.024570,1.023850,1.024495,NaN,1.024015
4116,20220720 01:40:00.595,1.02347,1.02352,8,0,1.023495,2022-07-20 01:40:00,1.023570,1.023885,1.023460,1.023630,NaN,1.023495
6866,20220720 02:54:53.368,1.02385,1.02389,1,0,1.023870,2022-07-20 02:54:53,1.023850,1.024275,1.023830,1.024195,NaN,1.023870
8774,20220720 03:54:58.365,1.02419,1.02425,1,0,1.024220,2022-07-20 03:54:58,1.024215,1.024345,1.024165,1.024330,NaN,1.024220
10380,20220720 04:44:56.632,1.02462,1.02465,1,0,1.024635,2022-07-20 04:44:56,1.024645,1.024765,1.024460,1.024510,NaN,1.024635
10644,20220720 04:54:55.323,1.02464,1.02467,6,0,1.024655,2022-07-20 04:54:55,1.024665,1.024705,1.024285,1.024375,NaN,1.024655
11930,20220720 05:24:59.244,1.02404,1.02408,1,0,1.024060,2022-07-20 05:24:59,1.024025,1.024720,1.023995,1.024595,NaN,1.024060
13093,20220720 05:50:00.300,1.02372,1.02378,2,0,1.023750,2022-07-20 05:50:00,1.023750,1.024020,1.023655,1.023870,NaN,1.023750
15274,20220720 06:15:00.778,1.02311,1.02316,1,0,1.023135,2022-07-20 06:15:00,1.023120,1.023635,1.023120,1.023500,NaN,1.023135
21297,20220720 07:10:00.295,1.02502,1.02505,1,0,1.025035,2022-07-20 07:10:00,1.025035,1.025655,1.024770,1.025635,NaN,1.025035


In [9]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
data['plot_start']  = '2022-07-20'
data['plot_stop']   = '2022-07-21'
data['things_to_plot'] = ['tick', 'positions']

In [10]:
plot_graph(data)